In [1]:
import numpy as np
from numpy import tanh,exp,sqrt
from numpy import max as npmax
from numpy import abs as npabs
from numpy import dot,diag,newaxis,zeros,array
from numpy.random import randn
from scipy.linalg import svd,qr,pinv2

In [2]:
def lc(x,alpha=1.0):
    return tanh(alpha*x)

In [3]:
def lcp(x,alpha=1.0):
    return alpha*(1.0 - tanh(alpha*x)**2)

In [4]:
def gauss(x,alpha=1.0):
    return x*exp(-(alpha*x**2)/2.0)

In [5]:
def gaussp(x,alpha=1.0):
    return (1.0 - alpha*x**2)*exp(-(alpha*x**2)/2.0)

In [6]:
def cube(x,alpha=1.0):
    return x**3

In [7]:
def cubep(x,alpha=0.0):
    return 3*x**2

In [8]:
def skew(x,alpha=0.0):
    return x**2

In [9]:
def skewp(x,alpha=0.0):
    return 2*x

In [10]:
def rowcenter(X):
    rowmeans = X.mean(axis=-1)
    return rowmeans, X - rowmeans[:, newaxis]

In [11]:
def whiteningmatrix(X,n):
    if(n > X.shape[0]):
        n = X.shape[0]
    U,D,Vt = svd(dot(X,X.T)/X.shape[1],full_matrices=False)
    return dot(diag(1.0/sqrt(D[0:n])),U[:,0:n].T),dot(U[:,0:n],diag(sqrt(D[0:n])))

In [12]:
def decorrelation_gs(w,W,p):
    w = w - (W[0:p,:]*dot(W[0:p,:],w.T)[:,newaxis]).sum(axis=0)
    w = w/sqrt(dot(w,w))
    return w

In [13]:
def decorrelation_witer(W):
    lim = 1.0
    tol = 1.0e-05
    W = W/(W**2).sum()
    while lim > tol:
        W1 = (3.0/2.0)*W - 0.5*dot(dot(W,W.T),W)
        lim = npmax(npabs(npabs(diag(dot(W1,W.T))) - 1.0))
        W = W1
    return W

In [14]:
def decorrelation_mdum(W):
    U,D,VT = svd(W)
    Y = dot(dot(U,diag(1.0/D)),U.T)
    return dot(Y,W)

In [15]:
def ica_def(X, tolerance, g, gprime, orthog, alpha, maxIterations, Winit):
    n,p = X.shape
    W = Winit
    # j is the index of the extracted component
    for j in xrange(n):
        w = Winit[j, :]
        it = 1
        lim = tolerance + 1
        while ((lim > tolerance) & (it < maxIterations)):
            wtx = dot(w, X)
            gwtx = g(wtx, alpha)
            g_wtx = gprime(wtx, alpha)
            w1 = (X * gwtx).mean(axis=1) - g_wtx.mean() * w
            w1 = decorrelation_gs(w1, W, j)
            lim = npabs(npabs((w1 * w).sum()) - 1.0)
            w = w1
            it = it + 1
        W[j, :] = w
    return W

In [16]:
def ica_par_fp(X, tolerance, g, gprime, orthog, alpha, maxIterations, Winit):
    n,p = X.shape
    W = orthog(Winit)
    lim = tolerance + 1
    it = 1
    while ((lim > tolerance) and (it < maxIterations)):
        wtx = dot(W,X)
        gwtx = g(wtx,alpha)
        g_wtx = gprime(wtx,alpha)
        W1 = dot(gwtx,X.T)/p - dot(diag(g_wtx.mean(axis=1)),W)
        W1 = orthog(W1)
        lim = npmax(npabs(npabs(diag(dot(W1,W.T))) - 1.0))
        W = W1
        it = it + 1
    return W

In [17]:
def ica(X, nSources=None, algorithm="parallel fp", decorrelation="mdum", nonlinearity="logcosh", alpha=1.0, maxIterations=500, tolerance=1e-05, Winit=None, scaled=True):
    algorithm_funcs = {'parallel fp':ica_par_fp, 'deflation':ica_def}
    orthog_funcs = {'mdum': decorrelation_mdum, 'witer': decorrelation_witer}

    if (alpha < 1) or (alpha > 2):
        raise ValueError("alpha must be in [1,2]")

    if nonlinearity == 'logcosh':
        g = lc
        gprime = lcp
    elif nonlinearity == 'exp':
        g = gauss
        gprime = gaussp
    elif nonlinearity == 'skew':
        g, gprime = skew, skewp
    else:
        g = cube
        gprime = cubep
    
    nmix,nsamp = X.shape
    
    # default is to do full-rank decomposition
    if nSources is None:
        nSources = nmix
    if Winit is None:    
        # start with a random orthogonal decomposition
        Winit = randn(nSources,nSources)

    # preprocessing (centering/whitening/pca)
    rowmeansX,X = rowcenter(X)
    Kw,Kd = whiteningmatrix(X,nSources)
    X = dot(Kw,X)

    # pass through kwargs
    kwargs = {'tolerance': tolerance,'g': g,'gprime': gprime,'orthog':orthog_funcs[decorrelation],'alpha': alpha,'maxIterations': maxIterations,'Winit': Winit}
    func = algorithm_funcs[algorithm]
    
    # run ICA
    W = func(X, **kwargs)

    # consruct the sources - means are not restored
    S = dot(W,X)

    # mixing matrix
    A = pinv2(dot(W,Kw))

    if scaled == True:
        S = S/S.std(axis=-1)[:,newaxis]
        A = A*S.std(axis=-1)[newaxis,:]
    
    return A,W,S

In [18]:
teste = np.array([[ 7.,  0.,  2.,  5.,  0.],
                  [ 8.,  0.,  1.,  0.,  0.],
                  [ 3.,  0.,  0.,  4.,  1.]])
    

In [19]:
a, b, c = ica(teste,2)

In [20]:
print (a)

[[-0.65237002  2.6879162 ]
 [-2.51424488  1.8484751 ]
 [ 0.38236518  1.52654912]]


In [21]:
print (b)

[[ 0.5181301   0.85530182]
 [-0.85530182  0.5181301 ]]


In [22]:
print (c)

[[-1.54802496 -0.05182671 -0.18702471  1.58967481  0.19720157]
 [ 1.22428706 -1.0482024  -0.55192096  1.1946827  -0.8188464 ]]
